In [10]:
!pip install openai vllm==0.8.2 pandas

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 131.9 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 81.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import string
from vllm import LLM, SamplingParams

# LLM
llm = LLM(model="iamjoon/llama-3-esg-binary-classifier-epoch1")

INFO 06-04 09:13:26 [__init__.py:239] Automatically detected platform cuda.


config.json:   0%|          | 0.00/745 [00:00<?, ?B/s]

INFO 06-04 09:13:32 [config.py:585] This model supports multiple tasks: {'embed', 'generate', 'classify', 'reward', 'score'}. Defaulting to 'generate'.
INFO 06-04 09:13:32 [config.py:1697] Chunked prefill is enabled with max_num_batched_tokens=8192.


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

INFO 06-04 09:15:26 [weight_utils.py:281] Time spent downloading weights for iamjoon/llama-3-esg-binary-classifier-epoch1: 109.385526 seconds


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 06-04 09:15:28 [loader.py:447] Loading weights took 2.01 seconds
INFO 06-04 09:15:28 [gpu_model_runner.py:1186] Model loading took 14.9596 GB and 112.088397 seconds
INFO 06-04 09:15:33 [backends.py:415] Using cache directory: /root/.cache/vllm/torch_compile_cache/2666d4ad2c/rank_0_0 for vLLM's torch.compile
INFO 06-04 09:15:33 [backends.py:425] Dynamo bytecode transform time: 5.21 s
INFO 06-04 09:15:36 [backends.py:132] Cache the graph of shape None for later use
INFO 06-04 09:15:53 [backends.py:144] Compiling a graph for general shape takes 18.93 s
INFO 06-04 09:16:05 [monitor.py:33] torch.compile takes 24.14 s in total
INFO 06-04 09:16:06 [kv_cache_utils.py:566] GPU KV cache size: 418,800 tokens
INFO 06-04 09:16:06 [kv_cache_utils.py:569] Maximum concurrency for 8,192 tokens per request: 51.12x
INFO 06-04 09:16:25 [gpu_model_runner.py:1534] Graph capturing finished in 19 secs, took 0.51 GiB
INFO 06-04 09:16:25 [core.py:151] init engine (profile, create kv cache, warmup model) to

In [3]:
# 프롬프트 템플릿 준비
template = string.Template("""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

${system_content}<|eot_id|><|start_header_id|>user<|end_header_id|>

${user_content}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
""")

system_prompt = '''1. 상장사의 뉴스를 보고 ESG 관련 기사인지 판단하시오.
2. ESG 각각에 대한 핵심 키워드는 아래의 10개 키워드를 참고해서 판단하시오.
- 환경 (Environmental) : 탄소 배출 감소 (Carbon Emission Reduction), 재생 에너지 사용 (Renewable Energy Usage), 자원 효율성 (Resource Efficiency), 폐기물 관리 (Waste Management), 친환경 제품 개발 (Eco-Friendly Product Development), 기후 변화 대응 (Climate Change Mitigation), 오염 방지 (Pollution Prevention), 생물 다양성 보호 (Biodiversity Protection), 물 관리 (Water Management)
순환 경제 (Circular Economy)
- 사회 (Social) : 노동 인권 보호 (Labor Rights Protection), 다양성 및 포용성 (Diversity and Inclusion), 직원 안전 및 건강 (Employee Health & Safety), 공정한 노동 관행 (Fair Labor Practices), 커뮤니티 참여 및 개발 (Community Engagement & Development), 제품 책임 및 안전 (Product Responsibility & Safety), 공급망 책임 (Supply Chain Responsibility), 데이터 보호 및 프라이버시 (Data Protection & Privacy), 인재 개발 및 교육 (Talent Development & Education), 고객 만족도 (Customer Satisfaction)
- 지배구조 (Governance) : 이사회 독립성 (Board Independence), 윤리적 경영 (Ethical Management), 주주 권리 보호 (Shareholder Rights Protection), 내부 통제 및 감사 (Internal Controls & Audits), 리스크 관리 (Risk Management), 투명한 정보 공개 (Transparent Disclosure), 기업 윤리 및 부패 방지 (Corporate Ethics & Anti-Corruption), 경영진 보상 구조 (Executive Compensation Structure), 컴플라이언스 (Regulatory Compliance), 이해관계자 참여 (Stakeholder Engagement)
3. 입력이 주어지면 근거를 작성하고 ESG 관련 기사이면 => True 아니면 => False를 작성합니다.
4. 답변은 항상 '=> True' 또는 '=> False'로 끝나야 하며 그 뒤에 아무 것도 적지마세요.
5. 지역의 정책이나 장소에 대한 내용이거나, 개인의 사건과 같이 특정 기업에 대한 이야기가 아닌 경우에는 아니라고 판단하세요. 저는 상장 '기업'의 내용에 집중하고 있습니다.
6. 기업의 경영권 분쟁과 같은 경우에는 단순히 개인의 이야기라고 볼 수 없습니다. 이 경우에도 ESG라고 판단할 수 있습니다.
7. 개인적인 부고의 기사는 ESG와 관련이 없다고 판단하세요.'''

user_prompt = '''LG에너지솔루션, 배터리 관리 분야 새 얼굴 '비 어라운드' 만들었다 LG에너지솔루션의 배터리 관리 토털 설루션(BMTS) 브랜드 'B.around'(비.어라운드) 로고. LG에너지솔루션 제공  잇단 화재로 전기차 배터리에 불안과 공포(전기차 포비아)가 확산한 가운데 LG에너지솔루션(LG엔솔)이 배터리 관리 토털 설루션(BMTS) 브랜드를 만들고 사업 확장에 나선다. 전기차 배터리 생산을 넘어 소프트웨어 부문 강화로 수익원을 넓히려는 움직임으로 풀이된다.  LG엔솔은 25일 '항상 고객의 곁에'(Be around your side)란 뜻의 BMTS 브랜드 'B.around'(비.어라운드)를 공개했다. 어떤 상황에서든 배터리 상태를 실시간 모니터링하는 LG엔솔 BMTS의 기술 경쟁력을 상징한다는 설명이다.  비.어라운드 제품에는 기존 배터리 관리 시스템(BMS)에 클라우드와 인공지능(AI) 기술을 결합해 화재방지 등을 위한 안전 진단, 퇴화·수명 예측 등을 더 쉽게 할 계획이다. 배터리 불량 유형을 사전에 진단하고 퇴화 상태를 점검하는 소프트웨어를 통해 오랫동안 쓸 수 있게 할 방침이다.  이달훈 LG에너지솔루션 BMS개발센터장 상무는 "오랜 기간 안전 진단 시스템을 위해 노력했고 이제는 배터리 관리 소프트웨어 사업으로 확장해 배터리의 건강한 사용을 위해 설루션을 제공하는 기업이 되려고 한다"고 말했다.  김청환 기자 chk@hankookilbo.com'''

# 메시지 준비
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

# 프롬프트 생성
prompt = template.safe_substitute({
    "system_content": messages[0]["content"],
    "user_content": messages[1]["content"]
})

In [4]:
prompt = prompt.strip() + '근거:'

In [5]:
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

1. 상장사의 뉴스를 보고 ESG 관련 기사인지 판단하시오.
2. ESG 각각에 대한 핵심 키워드는 아래의 10개 키워드를 참고해서 판단하시오.
- 환경 (Environmental) : 탄소 배출 감소 (Carbon Emission Reduction), 재생 에너지 사용 (Renewable Energy Usage), 자원 효율성 (Resource Efficiency), 폐기물 관리 (Waste Management), 친환경 제품 개발 (Eco-Friendly Product Development), 기후 변화 대응 (Climate Change Mitigation), 오염 방지 (Pollution Prevention), 생물 다양성 보호 (Biodiversity Protection), 물 관리 (Water Management)
순환 경제 (Circular Economy)
- 사회 (Social) : 노동 인권 보호 (Labor Rights Protection), 다양성 및 포용성 (Diversity and Inclusion), 직원 안전 및 건강 (Employee Health & Safety), 공정한 노동 관행 (Fair Labor Practices), 커뮤니티 참여 및 개발 (Community Engagement & Development), 제품 책임 및 안전 (Product Responsibility & Safety), 공급망 책임 (Supply Chain Responsibility), 데이터 보호 및 프라이버시 (Data Protection & Privacy), 인재 개발 및 교육 (Talent Development & Education), 고객 만족도 (Customer Satisfaction)
- 지배구조 (Governance) : 이사회 독립성 (Board Independence), 윤리적 경영 (Ethical Management), 주주 권리 

In [6]:
# 샘플링 파라미터 설정
sampling_params = SamplingParams(
    temperature=0,
    max_tokens=1024,
    stop=["\n<|end_of_text|>", "<|eot_id|>"],  # 문자열로 중지 토큰 지정
    repetition_penalty=1.0
)

# 생성
outputs = llm.generate([prompt], sampling_params)

# 결과 출력
for output in outputs:
    print()
    print(output.outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.18s/it, est. speed input: 458.46 toks/s, output: 82.23 toks/s]


 LG에너지솔루션의 배터리 관리 분야에서 '비 어라운드' 브랜드를 만든 것은 ESG와 관련이 있습니다. 첫째, 환경 측면에서 배터리 관리 소프트웨어를 통해 배터리의 안전 진단 및 퇴화 상태 점검을 통해 화재 방지와 오염 방지를 목표로 하고 있습니다. 이는 환경 보호와 관련된 활동입니다. 둘째, 사회적 측면에서는 고객의 안전을 최우선으로 고려하는 제품 개발로, 제품 책임 및 안전과 관련이 있습니다. 마지막으로, 지배구조 측면에서는 기술 경쟁력을 강화하고 고객의 곁에 있는 기업으로서의 이미지를 구축하려는 노력으로, 투명한 정보 공개와 윤리적 경영을 보여주는 사례입니다. 
=> True


In [11]:
import pandas as pd

df = pd.read_json('esg_binary_classification_test.json')

In [19]:
df

,instruction,input,output
0,"""월남쌈 대신 '비비고' 만두 먹고 CGV서 영화 봐요""…'베트남 젠지' 홀린 CJ...",,근거: 이 기사는 CJ그룹이 베트남에서 개최한 'CJ K FESTA'라는 K컬처 축...
1,"KDI ""한국경제 부진 지속…수출 위축에 내수도 둔화"" 국책연구원인 한국개발연구원,...",,"근거: 이 기사는 국책연구원인 한국개발연구원(KDI)의 한국 경제에 대한 분석으로,..."
2,개성 가득 이색 팝업 ’글로스튜디오’ 체험하는 배우 이이경 (서울=뉴스1) = BA...,,근거: 이 기사는 BAT로스만스의 이색 팝업 '글로 스튜디오' 오픈 행사와 배우 이...
3,"삼성전자 노조, 사측에 화해 손짓…""파업 상황 조속 해결"" 전국삼성전자노동조합 조합...",,근거: 해당 기사는 삼성전자의 노조와 사측 간의 파업 및 대화 제안에 관한 내용으로...
4,"‘클라우드’ 돕는 트레이너…‘AI’까지 넘본다 [천억클럽] 인공지능(AI) 시대, ...",,근거: 이 기사는 주로 클라우드 MSP(Managed Service Provider...
...,...,...,...
927,부산시 전기차 구매 보조금 지원대상 확대 [KBS 부산]부산시가 올해 상반기 전기자...,,근거: 부산시 전기차 구매 보조금 지원 대상 확대는 특정 기업과 관련된 이야기가 아...
928,KT&G “세계적인 의결권자문사의 이사회 제안 안건 ‘찬성 권고’ 대환영” 글로벌 ...,,근거: 해당 기사는 KT&G의 주주총회와 이사회 제안에 대한 의결권 자문사의 권고와...
929,서산 농작물 창고서 불…2백만 원 피해 [KBS 대전]오늘(27일) 오전 10시쯤 ...,,"근거: 이 기사는 서산의 농작물 창고에서 발생한 화재 사건에 관한 것으로, 특정 기..."
930,"현대차, 신형 코나 전기차 모델 첫 공개 자동요약 기사 제목과 주요 문장을 기반으로...",,근거: 현대차가 코나의 전기차 모델 '디 올 뉴 코나 일렉트릭'을 공개한 것은 ES...


In [12]:
inputs = df['instruction'].to_list()

In [13]:
def make_prompt(system_prompt, news_article):
    # 프롬프트 템플릿 준비
    template = string.Template("""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    
${system_content}<|eot_id|><|start_header_id|>user<|end_header_id|>
    
${user_content}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
""")
    
    user_prompt = news_article
    
    # 메시지 준비
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    # 프롬프트 생성
    prompt = template.safe_substitute({
        "system_content": messages[0]["content"],
        "user_content": messages[1]["content"]
    })

    prompt = prompt.strip() + "근거:"
    return prompt

In [14]:
prompt = make_prompt(system_prompt, inputs[0])
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    
1. 상장사의 뉴스를 보고 ESG 관련 기사인지 판단하시오.
2. ESG 각각에 대한 핵심 키워드는 아래의 10개 키워드를 참고해서 판단하시오.
- 환경 (Environmental) : 탄소 배출 감소 (Carbon Emission Reduction), 재생 에너지 사용 (Renewable Energy Usage), 자원 효율성 (Resource Efficiency), 폐기물 관리 (Waste Management), 친환경 제품 개발 (Eco-Friendly Product Development), 기후 변화 대응 (Climate Change Mitigation), 오염 방지 (Pollution Prevention), 생물 다양성 보호 (Biodiversity Protection), 물 관리 (Water Management)
순환 경제 (Circular Economy)
- 사회 (Social) : 노동 인권 보호 (Labor Rights Protection), 다양성 및 포용성 (Diversity and Inclusion), 직원 안전 및 건강 (Employee Health & Safety), 공정한 노동 관행 (Fair Labor Practices), 커뮤니티 참여 및 개발 (Community Engagement & Development), 제품 책임 및 안전 (Product Responsibility & Safety), 공급망 책임 (Supply Chain Responsibility), 데이터 보호 및 프라이버시 (Data Protection & Privacy), 인재 개발 및 교육 (Talent Development & Education), 고객 만족도 (Customer Satisfaction)
- 지배구조 (Governance) : 이사회 독립성 (Board Independence), 윤리적 경영 (Ethical Management), 주주

In [15]:
prompts = []
for input in inputs:
    prompts.append(make_prompt(system_prompt, input))

다수의 데이터 호출

In [16]:
results = llm.generate(prompts[:5], sampling_params)

Processed prompts: 100%|██████████| 5/5 [00:02<00:00,  2.08it/s, est. speed input: 3526.03 toks/s, output: 207.21 toks/s]


In [17]:
for result in results:
    print(result.outputs[0].text)
    print('--')

 이 기사는 CJ그룹이 베트남에서 K컬처 축제 'CJ K FESTA'를 개최한 내용으로, ESG와 관련된 핵심 키워드와는 직접적인 관련이 없습니다. 기사는 주로 CJ그룹의 브랜드 홍보 및 마케팅 활동에 관한 내용으로, 환경, 사회, 지배구조 측면에서의 ESG 활동이나 정책에 대한 언급이 없습니다. 따라서 ESG 관련 기사로 판단하기 어렵습니다. 
=> False
--
 이 기사는 한국개발연구원(KDI)의 경제 동향에 대한 내용으로, 특정 기업과 관련된 이야기가 아닙니다. 따라서 ESG와 관련이 없습니다. 
=> False
--
 이 기사는 BAT로스만스가 운영하는 이색 팝업 '글로스튜디오' 오픈에 대한 내용으로, ESG와 관련된 환경, 사회, 지배구조 측면의 핵심 키워드와는 관련이 없습니다. 이는 기업의 문화적 활동이나 마케팅 전략에 관한 내용으로, ESG의 핵심 요소와는 거리가 있습니다. 
=> False
--
 삼성전자 노조의 파업과 관련된 기사는 ESG와 관련이 있습니다. 사회적 측면에서 노동 인권 보호와 공정한 노동 관행이 핵심 요소로, 노조의 활동과 임금 교섭은 노동자들의 권리와 안전을 보호하는 데 중요한 역할을 합니다. 따라서 이 기사는 ESG의 사회적 책임과 관련이 있습니다. 
=> True
--
 베스핀글로벌의 클라우드 서비스 제공과 AI 기술 도입은 ESG와 관련이 있습니다. 첫째, 사회적 측면에서 데이터 관리와 AI 기술 도입은 인재 개발 및 교육과 관련이 있습니다. 베스핀글로벌은 데이터 전문 인력 양성 프로그램을 운영하고 있으며, 이는 인재 개발 및 교육의 일환으로 볼 수 있습니다. 둘째, 지배구조 측면에서 기업의 투명한 정보 공개와 리스크 관리가 중요합니다. 베스핀글로벌의 매출 성장과 적자 문제는 투명한 정보 공개와 리스크 관리의 필요성을 강조합니다. 따라서 이 기사는 ESG와 관련이 있습니다. 
=> True
--


In [18]:
results = llm.generate(prompts, sampling_params)

Processed prompts: 100%|██████████| 932/932 [01:00<00:00, 15.35it/s, est. speed input: 18881.39 toks/s, output: 1568.77 toks/s] 


In [20]:
labels = []
for label in df['output'].to_list():
    labels.append(label.split('=>')[1].strip())

preds = []
for pred in results:
    preds.append(pred.outputs[0].text.split('=>')[1].strip())

In [21]:
labels[:5]

['False', 'False', 'False', 'True', 'False']

In [22]:
preds[:5]

['False', 'False', 'False', 'True', 'True']

In [23]:
def calculate_metrics(labels, preds):
    """
    정확도와 F1 점수를 계산하는 함수 (문자열 "True"/"False" 사용).
    
    Args:
        labels (list): 실제 값 리스트 ("True"/"False").
        preds (list): 예측 값 리스트 ("True"/"False").

    Returns:
        dict: 정확도와 F1 점수를 포함한 결과.
    """
    # True Positive, False Positive, False Negative, True Negative 계산
    tp = sum(1 for l, p in zip(labels, preds) if l == "True" and p == "True")
    fp = sum(1 for l, p in zip(labels, preds) if l == "False" and p == "True")
    fn = sum(1 for l, p in zip(labels, preds) if l == "True" and p == "False")
    tn = sum(1 for l, p in zip(labels, preds) if l == "False" and p == "False")
    
    # Precision, Recall 계산
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    # F1 Score 계산
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # Accuracy 계산
    accuracy = (tp + tn) / len(labels) if len(labels) > 0 else 0
    
    return {"accuracy": accuracy, "f1_score": f1_score}

# 함수 호출
metrics = calculate_metrics(labels, preds)
print(metrics)

{'accuracy': 0.8626609442060086, 'f1_score': 0.8785578747628083}


In [24]:
df['label'] = labels
df['pred'] = preds

In [25]:
pred_reasons = ['근거:' + r.outputs[0].text for r in results]
df['pred_reasons'] = pred_reasons 

In [26]:
df

,instruction,input,output,label,pred,pred_reasons
0,"""월남쌈 대신 '비비고' 만두 먹고 CGV서 영화 봐요""…'베트남 젠지' 홀린 CJ...",,근거: 이 기사는 CJ그룹이 베트남에서 개최한 'CJ K FESTA'라는 K컬처 축...,False,False,근거: 이 기사는 CJ그룹이 베트남에서 K컬처 축제 'CJ K FESTA'를 개최한...
1,"KDI ""한국경제 부진 지속…수출 위축에 내수도 둔화"" 국책연구원인 한국개발연구원,...",,"근거: 이 기사는 국책연구원인 한국개발연구원(KDI)의 한국 경제에 대한 분석으로,...",False,False,"근거: 이 기사는 한국개발연구원(KDI)의 경제 동향에 대한 내용으로, 특정 기업과..."
2,개성 가득 이색 팝업 ’글로스튜디오’ 체험하는 배우 이이경 (서울=뉴스1) = BA...,,근거: 이 기사는 BAT로스만스의 이색 팝업 '글로 스튜디오' 오픈 행사와 배우 이...,False,False,근거: 이 기사는 BAT로스만스가 운영하는 이색 팝업 '글로스튜디오' 오픈에 대한 ...
3,"삼성전자 노조, 사측에 화해 손짓…""파업 상황 조속 해결"" 전국삼성전자노동조합 조합...",,근거: 해당 기사는 삼성전자의 노조와 사측 간의 파업 및 대화 제안에 관한 내용으로...,True,True,근거: 삼성전자 노조의 파업과 관련된 기사는 ESG와 관련이 있습니다. 사회적 측면...
4,"‘클라우드’ 돕는 트레이너…‘AI’까지 넘본다 [천억클럽] 인공지능(AI) 시대, ...",,근거: 이 기사는 주로 클라우드 MSP(Managed Service Provider...,False,True,근거: 베스핀글로벌의 클라우드 서비스 제공과 AI 기술 도입은 ESG와 관련이 있습...
...,...,...,...,...,...,...
927,부산시 전기차 구매 보조금 지원대상 확대 [KBS 부산]부산시가 올해 상반기 전기자...,,근거: 부산시 전기차 구매 보조금 지원 대상 확대는 특정 기업과 관련된 이야기가 아...,False,False,"근거: 부산시의 전기차 구매 보조금 지원 확대는 특정 기업과 관련된 내용이 아니며,..."
928,KT&G “세계적인 의결권자문사의 이사회 제안 안건 ‘찬성 권고’ 대환영” 글로벌 ...,,근거: 해당 기사는 KT&G의 주주총회와 이사회 제안에 대한 의결권 자문사의 권고와...,True,True,근거: KT&G의 이사회 제안에 대한 글래스루이스의 찬성 권고는 ESG와 관련이 있...
929,서산 농작물 창고서 불…2백만 원 피해 [KBS 대전]오늘(27일) 오전 10시쯤 ...,,"근거: 이 기사는 서산의 농작물 창고에서 발생한 화재 사건에 관한 것으로, 특정 기...",False,False,"근거: 이 기사는 특정 기업과 관련된 내용이 아니며, 지역의 농작물 창고에서 발생한..."
930,"현대차, 신형 코나 전기차 모델 첫 공개 자동요약 기사 제목과 주요 문장을 기반으로...",,근거: 현대차가 코나의 전기차 모델 '디 올 뉴 코나 일렉트릭'을 공개한 것은 ES...,True,True,근거: 현대차가 신형 코나 전기차 모델을 공개한 것은 ESG와 관련이 있습니다. 환...


In [27]:
idx = 0

print('뉴스 기사')
print(df['instruction'].loc[idx])
print('--' * 20)
print('정답')
print(df['output'].loc[idx])
print('--' * 20)
print('예측')
print(df['pred_reasons'].loc[idx])

뉴스 기사
"월남쌈 대신 '비비고' 만두 먹고 CGV서 영화 봐요"…'베트남 젠지' 홀린 CJ CJ그룹은 지난 10일부터 9월 1일까지 베트남에서 K컬처 축제 'CJ K FESTA'를 연다.ⓒ 뉴스1/김진희 기자.  (호찌민=뉴스1) 김진희 기자 = "CJ(001040) 로고가 화려하고 예쁩니다. CGV를 통해 CJ를 알게 됐는데 젊고 역동적이어서 '젠지' 세대에 적합한 이미지라는 생각이 듭니다." -화이빠오(24·남)  지난 15~16일(현지시간) 베트남 호찌민 일대에서 열린 'CJ K FESTA'를 찾은 베트남 현지인들은 극찬을 쏟아냈다.  CJ그룹은 베트남에 진출한 계열사를 총동원해 CJ K FESTA를 올해 처음으로 개최했다. 이번 축제는 △K푸드 Week(8월 10~16일) △K스포츠 Week(8월 17~23일) △K무비 Week(8월 24일~9월 1일) 등 3가지 테마로 구성됐다.  CJ는 호찌민 소재 유명 마트와 쇼핑몰 내 CGV 매장에서 푸드쇼를 운영하며 브랜드 홍보에 적극 나섰다.  CJ그룹은 지난 10일부터 9월 1일까지 베트남에서 K컬처 축제 'CJ K FESTA'를 연다.ⓒ 뉴스1/김진희 기자.  15일 오후 호찌민 소재 메가마켓에는 더운 날씨임에도 마트 외부 공간에 사람들로 북적였다. CJ 제품을 선보이는 푸드트럭을 구경하기 위해 찾은 이들이었다. 한국과 달리 베트남에서는 푸드트럭이 생소하다고 한다.  이날 CJ는 CJ제일제당 비비고의 해산물 만두, 호떡, 김스낵과 뚜레쥬르의 빵을 선보였다. CJ제일제당의 자회사인 베트남 현지 냉동식품기업 '까우제'(Cautre)의 제품도 만나볼 수 있었다.  호찌민 시민 화이빠오는 "예전에 만두를 많이 먹어 봤지만 비비고 만두는 독특하고 맛있다"며 "특히 푸드트럭을 처음 봐서 신기한데, 이렇게 화려하게 시식·광고하는 건 처음 본다"고 말했다.  CJ그룹은 지난 10일부터 9월 1일까지 베트남에서 K컬처 축제 'CJ K FESTA'를 연다.ⓒ 뉴스1/김진희 기자.  아이와 함께 푸드트럭을 방문한 주부 흐

In [28]:
df[df['label']!=df['pred']]

,instruction,input,output,label,pred,pred_reasons
4,"‘클라우드’ 돕는 트레이너…‘AI’까지 넘본다 [천억클럽] 인공지능(AI) 시대, ...",,근거: 이 기사는 주로 클라우드 MSP(Managed Service Provider...,False,True,근거: 베스핀글로벌의 클라우드 서비스 제공과 AI 기술 도입은 ESG와 관련이 있습...
16,"""한 달 빨라졌다""...한화그룹 14개 계열사 정기 임원 인사, 51명 임원 승진 ...",,"근거: 이 기사는 한화그룹의 14개 계열사 정기 임원 인사에 관한 내용으로, 인사가...",True,False,"근거: 이 기사는 한화그룹의 14개 계열사 정기 임원 인사에 대한 내용으로, ESG..."
34,"원유석 아시아나항공 대표 직무대행, 부사장 승진 아시아나항공은 15일 원유석 대표이...",,근거: 아시아나항공의 임원 인사 단행은 ESG의 지배구조(Governance)와 관...,True,False,근거: 아시아나항공의 임원 인사와 관련된 기사는 ESG와 직접적인 관련이 없습니다....
46,현대차 ‘2024 아·태 세이버 어워즈’ 수상[포토뉴스] 이 글자크기로 변경됩니다....,,근거: 현대차가 '2024 아·태 세이버 어워즈'를 수상했다는 내용은 구체적으로 E...,False,True,근거: 현대차가 '2024 아·태 세이버 어워즈'를 수상한 것은 ESG와 관련이 있...
61,세계 1위 인구 대국 인도가 꼽은 ‘올해의 차’는 기아 ‘카렌스’ 기아 미니밴 ‘카...,,근거: 이 기사는 기아자동차의 제품이 인도에서 '올해의 차'와 '그린카' 부문에서 ...,False,True,근거: 기아가 인도에서 '올해의 차'와 '그린카' 부문에서 수상한 것은 ESG와 관...
...,...,...,...,...,...,...
859,"LG전자, 일체형 로봇청소기 첫 출시… 中에 도전장 LG전자가 자사 첫 올인원 로봇...",,근거: LG전자의 일체형 로봇청소기 출시 기사는 ESG와 직접적인 관련이 없습니다....,False,True,"근거: LG전자가 일체형 로봇청소기를 출시한 것은 ESG와 관련이 있습니다. 첫째,..."
889,"동원산업, 보령바이오파마 인수 불발…""실사 우선권 해지"" 자동요약 기사 제목과 주요...",,"근거: 동원산업의 보령바이오파마 인수 불발은 기업 간의 인수합병 관련 소식으로, E...",False,True,근거: 동원산업이 보령바이오파마 인수를 포기한 것은 기업의 인수 및 합병과 관련된 ...
898,"티웨이항공 싱가포르발 항공기, 기체 안전점검 이유로 이륙 직후 회항 싱가포르를 떠나...",,근거: 티웨이항공의 항공기가 안전점검을 이유로 회항한 사건은 직접적인 ESG 관련 ...,False,True,"근거: 이 기사는 티웨이항공의 항공기 안전점검으로 인한 회항과 관련된 내용으로, E..."
907,"'삐약이' 신유빈, 바나나맛 우유 광고 찍고 '빙그레' 탁구 국가대표 선수 신유빈이...",,"근거: 이 기사는 기업 빙그레와 관련된 내용으로, 신유빈 선수가 출연한 바나나맛우유...",True,False,"근거: 이 기사는 '빙그레'라는 기업의 광고 촬영과 관련된 내용으로, ESG와 관련..."


In [29]:
idx = 16

print('뉴스 기사')
print(df['instruction'].loc[idx])
print('--' * 20)
print('정답')
print(df['output'].loc[idx])
print('--' * 20)
print('예측')
print(df['pred_reasons'].loc[idx])

뉴스 기사
"한 달 빨라졌다"...한화그룹 14개 계열사 정기 임원 인사, 51명 임원 승진 한화그룹 기업 이미지(CI). 한화그룹 제공  한화그룹이 14개 계열사의 정기 임원 인사를 단행했다. 인사 시기가 예년보다 한 달가량 앞당겨진 것으로 글로벌 경영 환경이 급변하는 데 빠르게 대응하려는 의도란 설명이다.  한화그룹은 27일 지주회사 격인 ㈜한화를 비롯해 총 14개사의 정기 임원 인사를 단행했다고 밝혔다. 한화에어로스페이스, 한화솔루션, 한화오션, 한화시스템, 한화에너지, 한화임팩트, 한화토탈에너지스, 한화파워시스템, 한화엔진, 한화첨단소재, 한화이센셜, 한화비전, 한화정밀기계 등이다.  이번 인사로 총 51명이 임원으로 승진했다. 승진 일자는 10월 1일이다.  한화그룹의 모태이자 주력 사업인 방위 산업을 담당하는 계열사 중 한화에어로스페이스가 가장 많은 12명의 임원을 배출했다. 한화시스템도 3명을 임원으로 승진시켰다. 한화솔루션은 7명의 신임 임원을 발탁했다. 한화솔루션은 현재의 경영 위기를 극복하고 사업 전환을 가속하기 위해 대표이사 인사에 이어 임원 인사도 앞당겨진 것이라고 설명했다.  조선업 부문인 한화오션(옛 대우조선해양)도 이날 임원 7명 승진 인사를 단행했다. 한화오션은 공정 안정화와 차별화된 원가 경쟁력 확보를 위해 현장 경험과 전문성을 겸비한 생산 분야 전문 인력 발탁에 이번 인사의 중점을 뒀다고 밝혔다. 임원 승진자 7명의 소속은 연구개발(R&D) 1명, 설계 2명, 생산 4명 등이다.  이 밖에 한화에너지, 한화임팩트, 한화토탈에너지스, 한화파워시스템, 한화엔진, 한화비전, 한화정밀기계 등 7개사가 총 13명을 신규 임원으로 승진시켰다. 한화솔루션의 자회사인 한화첨단소재와 한화이센셜도 각각 한 명의 임원 승진 인사를 발표했다.  이날 인사가 난 회사는 대부분 김승연 회장을 잇는 한화그룹 오너 3세 삼형제(김동관 김동원 김동선) 가운데 장남 김동관 부회장이 이끄는 곳이란 공통점이 있다. 예년보다 빨라진 인사에서 그의 경영 스타일을 엿볼